In [15]:
import pandas as pd
import numpy as np
import datetime as dt
import requests
from io import StringIO
import json
from investiny import historical_data, search_assets
import plotly.express as px
import plotly.graph_objects as go
import psycopg2
from config import config
from sqlalchemy import create_engine

## CPI data

### CPI data up to 2019

In [5]:
# up to 2023
# url = 'https://webapi.bps.go.id/v1/api/list/model/data/lang/ind/domain/0000/var/1709/key/WebAPI_KEY'
# up to 2019
url = 'https://webapi.bps.go.id/v1/api/list/model/data/lang/ind/domain/0000/var/2/key/c2de5ecae1ffee1cbfb2a8c6edeb9994'
response = requests.get(url)
response.raise_for_status()
fetched = response.json()

fixed = json.dumps(fetched)
data = json.loads(fixed)

In [6]:
datacontent = data['datacontent']

In [7]:
data.keys()

dict_keys(['status', 'data-availability', 'var', 'turvar', 'labelvervar', 'vervar', 'tahun', 'turtahun', 'metadata', 'datacontent'])

In [8]:
data['turvar']

[{'val': 0, 'label': 'Tidak Ada'}]

In [9]:
data['tahun']

[{'val': 79, 'label': '1979'},
 {'val': 80, 'label': '1980'},
 {'val': 81, 'label': '1981'},
 {'val': 82, 'label': '1982'},
 {'val': 83, 'label': '1983'},
 {'val': 84, 'label': '1984'},
 {'val': 85, 'label': '1985'},
 {'val': 86, 'label': '1986'},
 {'val': 87, 'label': '1987'},
 {'val': 88, 'label': '1988'},
 {'val': 89, 'label': '1989'},
 {'val': 90, 'label': '1990'},
 {'val': 91, 'label': '1991'},
 {'val': 92, 'label': '1992'},
 {'val': 93, 'label': '1993'},
 {'val': 94, 'label': '1994'},
 {'val': 95, 'label': '1995'},
 {'val': 96, 'label': '1996'},
 {'val': 97, 'label': '1997'},
 {'val': 98, 'label': '1998'},
 {'val': 99, 'label': '1999'},
 {'val': 100, 'label': '2000'},
 {'val': 101, 'label': '2001'},
 {'val': 102, 'label': '2002'},
 {'val': 103, 'label': '2003'},
 {'val': 104, 'label': '2004'},
 {'val': 105, 'label': '2005'},
 {'val': 106, 'label': '2006'},
 {'val': 107, 'label': '2007'},
 {'val': 108, 'label': '2008'},
 {'val': 109, 'label': '2009'},
 {'val': 110, 'label': '2010'

In [10]:
year_mapping = {item['val']:item['label'] for item in data['tahun']}

In [11]:
year_mapping

{79: '1979',
 80: '1980',
 81: '1981',
 82: '1982',
 83: '1983',
 84: '1984',
 85: '1985',
 86: '1986',
 87: '1987',
 88: '1988',
 89: '1989',
 90: '1990',
 91: '1991',
 92: '1992',
 93: '1993',
 94: '1994',
 95: '1995',
 96: '1996',
 97: '1997',
 98: '1998',
 99: '1999',
 100: '2000',
 101: '2001',
 102: '2002',
 103: '2003',
 104: '2004',
 105: '2005',
 106: '2006',
 107: '2007',
 108: '2008',
 109: '2009',
 110: '2010',
 111: '2011',
 112: '2012',
 113: '2013',
 114: '2014',
 115: '2015',
 116: '2016',
 117: '2017',
 118: '2018',
 119: '2019'}

In [13]:
max(year_mapping)

119

In [14]:
data['turtahun']

[{'val': 1, 'label': 'Januari'},
 {'val': 2, 'label': 'Februari'},
 {'val': 3, 'label': 'Maret'},
 {'val': 4, 'label': 'April'},
 {'val': 5, 'label': 'Mei'},
 {'val': 6, 'label': 'Juni'},
 {'val': 7, 'label': 'Juli'},
 {'val': 8, 'label': 'Agustus'},
 {'val': 9, 'label': 'September'},
 {'val': 10, 'label': 'Oktober'},
 {'val': 11, 'label': 'November'},
 {'val': 12, 'label': 'Desember'}]

In [15]:
month_mapping = {item['val']:item['label'] for item in data['turtahun']}

In [16]:
filter_keys = [i+str(num) for num in range(1,13) for i in ['999920' + str(num) for num in range(117, 120)]]

In [17]:
len(filter_keys)

36

In [18]:
filtered = {key: value for key, value in datacontent.items() if key in filter_keys}

In [19]:
filtered

{'9999201171': 127.94,
 '9999201172': 128.24,
 '9999201173': 128.22,
 '9999201174': 128.33,
 '9999201175': 128.83,
 '9999201176': 129.72,
 '9999201177': 130,
 '9999201178': 129.91,
 '9999201179': 130.08,
 '99992011710': 130.09,
 '99992011711': 130.35,
 '99992011712': 131.28,
 '9999201181': 132.1,
 '9999201182': 132.32,
 '9999201183': 132.58,
 '9999201184': 132.71,
 '9999201185': 132.99,
 '9999201186': 133.77,
 '9999201187': 134.14,
 '9999201188': 134.07,
 '9999201189': 133.83,
 '99992011810': 134.2,
 '99992011811': 134.56,
 '99992011812': 135.39,
 '9999201191': 135.83,
 '9999201192': 135.72,
 '9999201193': 135.87,
 '9999201194': 136.47,
 '9999201195': 137.4,
 '9999201196': 138.16,
 '9999201197': 138.59,
 '9999201198': 138.75,
 '9999201199': 138.37,
 '99992011910': 138.4,
 '99992011911': 138.6,
 '99992011912': 139.07}

In [20]:
year_mapping

{79: '1979',
 80: '1980',
 81: '1981',
 82: '1982',
 83: '1983',
 84: '1984',
 85: '1985',
 86: '1986',
 87: '1987',
 88: '1988',
 89: '1989',
 90: '1990',
 91: '1991',
 92: '1992',
 93: '1993',
 94: '1994',
 95: '1995',
 96: '1996',
 97: '1997',
 98: '1998',
 99: '1999',
 100: '2000',
 101: '2001',
 102: '2002',
 103: '2003',
 104: '2004',
 105: '2005',
 106: '2006',
 107: '2007',
 108: '2008',
 109: '2009',
 110: '2010',
 111: '2011',
 112: '2012',
 113: '2013',
 114: '2014',
 115: '2015',
 116: '2016',
 117: '2017',
 118: '2018',
 119: '2019'}

In [21]:
filtered.keys()

dict_keys(['9999201171', '9999201172', '9999201173', '9999201174', '9999201175', '9999201176', '9999201177', '9999201178', '9999201179', '99992011710', '99992011711', '99992011712', '9999201181', '9999201182', '9999201183', '9999201184', '9999201185', '9999201186', '9999201187', '9999201188', '9999201189', '99992011810', '99992011811', '99992011812', '9999201191', '9999201192', '9999201193', '9999201194', '9999201195', '9999201196', '9999201197', '9999201198', '9999201199', '99992011910', '99992011911', '99992011912'])

In [22]:
df = pd.DataFrame.from_dict(filtered,orient='index').reset_index().rename(columns={'index':'Code',0:'CPI'})

In [23]:
df.head()

,Code,CPI
0,9999201171,127.94
1,9999201172,128.24
2,9999201173,128.22
3,9999201174,128.33
4,9999201175,128.83


In [24]:
df['Code'] = df['Code'].apply(lambda x: x[6:])

In [25]:
df.tail()

,Code,CPI
31,1198,138.75
32,1199,138.37
33,11910,138.40
34,11911,138.60
35,11912,139.07


In [26]:
df['year_code'] = df['Code'].str[:3].astype('int32')
df['month_code'] = df['Code'].str[3:].astype('int32')
df['Year'] = df['year_code'].map(year_mapping)
df['Month'] = df['month_code'].map(month_mapping)

### CPI data up to 2023

In [27]:
url = 'https://webapi.bps.go.id/v1/api/list/model/data/lang/ind/domain/0000/var/1709/key/c2de5ecae1ffee1cbfb2a8c6edeb9994'
response = requests.get(url)
response.raise_for_status()
fetched = response.json()

fixed = json.dumps(fetched)
data2 = json.loads(fixed)
datacontent2 = data['datacontent']

In [28]:
datacontent2

{'999920791': 118.33,
 '999920792': 121.1,
 '999920793': 121.77,
 '999920794': 125.45,
 '999920795': 129.27,
 '999920796': 132.27,
 '999920797': 135.58,
 '999920798': 138.75,
 '999920799': 139.78,
 '9999207910': 141.03,
 '9999207911': 141.84,
 '9999207912': 143.07,
 '999920801': 144.77,
 '999920802': 146.82,
 '999920803': 147.14,
 '999920804': 148.67,
 '999920805': 154.31,
 '999920806': 156.61,
 '999920807': 158.33,
 '999920808': 160.21,
 '999920809': 160.78,
 '9999208010': 163.49,
 '9999208011': 167.12,
 '9999208012': 167.55,
 '999920811': 169.75,
 '999920812': 170.94,
 '999920813': 172.14,
 '999920814': 173.74,
 '999920815': 174,
 '999920816': 174.73,
 '999920817': 176.83,
 '999920818': 177.74,
 '999920819': 177.4,
 '9999208110': 179.48,
 '9999208111': 178.91,
 '9999208112': 179.82,
 '999920821': 188.28,
 '999920822': 189.29,
 '999920823': 189.63,
 '999920824': 189.52,
 '999920825': 189.86,
 '999920826': 190.49,
 '999920827': 192.6,
 '999920828': 191.72,
 '999920829': 193.41,
 '99992

In [29]:
data2.keys()

dict_keys(['status', 'data-availability', 'var', 'turvar', 'labelvervar', 'vervar', 'tahun', 'turtahun', 'metadata', 'datacontent'])

In [30]:
data2['var']

[{'val': 1709,
  'label': 'Indeks Harga Konsumen 90 Kota (Umum)',
  'unit': '',
  'subj': 'Inflasi',
  'def': '',
  'decimal': '2',
  'note': '&lt;p&gt;&lt;br /&gt;&lt;/p&gt;&lt;p&gt;Mulai Tahun 2020, digunakan tahun dasar 2018, IHK dihitung berdasarkan pola konsumsi hasil SBH di 90 kota tahun 2018 (2018=100)&lt;br /&gt;&lt;/p&gt;&lt;p&gt;&lt;br /&gt;&lt;/p&gt;'}]

In [31]:
data2['turvar']

[{'val': 0, 'label': 'Tidak Ada'}]

In [32]:
data2['vervar']

[{'val': 1107, 'label': 'KOTA MEULABOH'},
 {'val': 1171, 'label': 'KOTA BANDA ACEH'},
 {'val': 1174, 'label': 'KOTA LHOKSEUMAWE'},
 {'val': 1271, 'label': 'KOTA SIBOLGA'},
 {'val': 1273, 'label': 'KOTA PEMATANG SIANTAR'},
 {'val': 1275, 'label': 'KOTA MEDAN'},
 {'val': 1277, 'label': 'KOTA PADANGSIDIMPUAN'},
 {'val': 1278, 'label': 'KOTA GUNUNGSITOLI'},
 {'val': 1371, 'label': 'KOTA PADANG'},
 {'val': 1375, 'label': 'KOTA BUKITTINGGI'},
 {'val': 1403, 'label': 'TEMBILAHAN'},
 {'val': 1471, 'label': 'KOTA PEKANBARU'},
 {'val': 1473, 'label': 'KOTA DUMAI'},
 {'val': 1509, 'label': 'BUNGO'},
 {'val': 1571, 'label': 'KOTA JAMBI'},
 {'val': 1671, 'label': 'KOTA PALEMBANG'},
 {'val': 1674, 'label': 'KOTA LUBUKLINGGAU'},
 {'val': 1771, 'label': 'KOTA BENGKULU'},
 {'val': 1871, 'label': 'KOTA BANDAR LAMPUNG'},
 {'val': 1872, 'label': 'KOTA METRO'},
 {'val': 1902, 'label': 'TANJUNG PANDAN'},
 {'val': 1971, 'label': 'KOTA PANGKAL PINANG'},
 {'val': 2171, 'label': 'KOTA BATAM'},
 {'val': 2172, 'l

In [33]:
vervar = next(item['val'] for item in data['vervar'] if item['label'].upper() == 'INDONESIA')

In [34]:
var = next(item['val'] for item in data2['var'])

In [35]:
turvar = next(item['val'] for item in data2['turvar'])

In [36]:
turvar

0

In [37]:
data2['tahun']

[{'val': 120, 'label': '2020'},
 {'val': 121, 'label': '2021'},
 {'val': 122, 'label': '2022'},
 {'val': 123, 'label': '2023'}]

In [38]:
data2['turtahun']

[{'val': 1, 'label': 'Januari'},
 {'val': 2, 'label': 'Februari'},
 {'val': 3, 'label': 'Maret'},
 {'val': 4, 'label': 'April'},
 {'val': 5, 'label': 'Mei'},
 {'val': 6, 'label': 'Juni'},
 {'val': 7, 'label': 'Juli'},
 {'val': 8, 'label': 'Agustus'},
 {'val': 9, 'label': 'September'},
 {'val': 10, 'label': 'Oktober'},
 {'val': 11, 'label': 'November'},
 {'val': 12, 'label': 'Desember'}]

### JSON content and structure is pretty much similar to our previous data, might as well create a function for it

In [3]:
API_key = 'c2de5ecae1ffee1cbfb2a8c6edeb9994'

In [97]:
def get_monthly_cpi(url):
    api_key = API_key
    url = f'{url}/{api_key}'
    response = requests.get(url)
    response.raise_for_status()
    fetched = response.json()
    fixed = json.dumps(fetched)
    data = json.loads(fixed)
    
    vervar = next(item['val'] for item in data['vervar'] if item['label'].upper() == 'INDONESIA')
    var = next(item['val'] for item in data['var'])
    turvar = next(item['val'] for item in data['turvar'])
    code = str(vervar)+str(var)+str(turvar)
    
    datacontent = data['datacontent']
    year_mapping = {item['val']:item['label'] for item in data['tahun']}
    month_mapping = {item['val']:item['label'] for item in data['turtahun']}
    last_four_years = [key for key in year_mapping.keys() if key >= max(year_mapping)-3]   
    filter_keys = [i+str(num) for num in range(1,13) for i in [code + str(num) for num in last_four_years]]
    filtered = {key: value for key, value in datacontent.items() if key in filter_keys}
    df = pd.DataFrame.from_dict(filtered,orient='index').reset_index().rename(columns={'index':'year_month',0:'CPI'})
    
    df['year_month'] = df['year_month'].apply(lambda x: x[len(code):])
    df['year_code'] = df['year_month'].str[:3].astype('int32')
    df['month_code'] = df['year_month'].str[3:].astype('int32')
    df['Year'] = df['year_code'].map(year_mapping)
    df['Month'] = df['month_code'].map(month_mapping)
    return df

In [98]:
df_2019 = get_monthly_cpi('https://webapi.bps.go.id/v1/api/list/model/data/lang/ind/domain/0000/var/2/key')

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [42]:
df_2023 = get_monthly_cpi('https://webapi.bps.go.id/v1/api/list/model/data/lang/ind/domain/0000/var/1709/key')

In [43]:
df_2024 = get_monthly_cpi('https://webapi.bps.go.id/v1/api/list/model/data/lang/ind/domain/0000/var/2261/key')

In [44]:
df_2024.tail()

,year_month,CPI,year_code,month_code,Year,Month
0,1241,105.19,124,1,2024,Januari
1,1242,105.58,124,2,2024,Februari
2,1243,106.13,124,3,2024,Maret
3,1244,106.40,124,4,2024,April


### Gausah di-include, cuma untuk cek

In [ ]:
filter_keys2 = [i + str(num) for num in range(1,13) for i in ['110717090' + str(num) for num in range(120,124)]]

In [ ]:
len(filter_keys2)

In [ ]:
filtered2 = {key: value for key, value in datacontent2.items() if key in filter_keys2}

In [ ]:
df2 = pd.DataFrame.from_dict(filtered2,orient='index').reset_index().rename(columns={'index':'Code',0:'CPI'})

In [ ]:
year_mapping2 = {item['val']:item['label'] for item in data2['tahun']}

In [ ]:
df2['Code'] = df2['Code'].apply(lambda x: x[9:])
df2['year_code'] = df2['Code'].str[:3].astype('int32')
df2['month_code'] = df2['Code'].str[3:].astype('int32')

df2['Year'] = df2['year_code'].map(year_mapping2)
df2['Month'] = df2['month_code'].map(month_mapping)

In [ ]:
df2

### Union all CPI data

In [45]:
cpi = pd.concat([df_2019,df_2023,df_2024],ignore_index=True)

In [46]:
cpi.head()

,year_month,CPI,year_code,month_code,Year,Month
0,1161,123.62,116,1,2016,Januari
1,1162,123.51,116,2,2016,Februari
2,1163,123.75,116,3,2016,Maret
3,1164,123.19,116,4,2016,April
4,1165,123.48,116,5,2016,Mei


## Other data

In [48]:
# search_results = search_assets(query="BTC", type="ETF", exchange="IDX")
# # investing_id = int(search_results[0]["ticker"]) # Assuming the first entry is the desired one (top result in Investing.com)

# # data = historical_data(investing_id=investing_id, from_date="09/01/2022", to_date="10/01/2022")
# search_results

In [77]:
btc = pd.read_csv('csv_files/Data Historis BTC_IDR BTC Indonesia 2017-2024.csv')
btc.head()

,Tanggal,Terakhir,Pembukaan,Tertinggi,Terendah,Vol.,Perubahan%
0,31/03/2024,1.129.259.008,1.113.600.000,1.129.260.032,1.113.600.000,"0,02K","1,41%"
1,30/03/2024,1.113.600.000,1.117.326.976,1.120.000.000,1.113.500.032,"0,02K","-0,28%"
2,29/03/2024,1.116.734.976,1.130.334.976,1.130.855.040,1.105.056.000,"0,03K","-1,20%"
3,28/03/2024,1.130.334.976,1.109.442.944,1.135.718.016,1.100.519.936,"0,04K","1,83%"
4,27/03/2024,1.109.984.000,1.114.294.016,1.130.000.000,1.091.016.960,"0,07K","-0,38%"


In [50]:
ihsg = pd.read_csv('Data Historis Jakarta Stock Exchange Composite 2017-2024.csv')
ihsg.head()

,Tanggal,Terakhir,Pembukaan,Tertinggi,Terendah,Vol.,Perubahan%
0,28/03/2024,"7.288,81","7.289,16","7.313,01","7.244,95","12,41B","-0,29%"
1,27/03/2024,"7.310,09","7.364,23","7.375,40","7.292,80","12,53B","-0,75%"
2,26/03/2024,"7.365,66","7.337,66","7.371,84","7.330,46","13,84B","-0,16%"
3,25/03/2024,"7.377,76","7.339,52","7.377,76","7.316,93","12,43B","0,38%"
4,22/03/2024,"7.350,15","7.327,51","7.350,15","7.318,44","12,37B","0,16%"


In [51]:
usd = pd.read_csv('Data Historis USD_IDR 2017-2024.csv')
usd.head()

,Tanggal,Terakhir,Pembukaan,Tertinggi,Terendah,Vol.,Perubahan%
0,29/03/2024,"15.880,0","15.891,0","15.909,3","15.858,5",NaN,"0,19%"
1,28/03/2024,"15.850,0","15.880,0","15.894,0","15.855,0",NaN,"0,00%"
2,27/03/2024,"15.850,0","15.830,0","15.865,0","15.818,5",NaN,"0,41%"
3,26/03/2024,"15.785,0","15.800,0","15.810,0","15.780,0",NaN,"-0,06%"
4,25/03/2024,"15.795,0","15.785,0","15.808,5","15.775,0",NaN,"0,13%"


In [52]:
gold = pd.read_csv('Data Historis GAU_IDR.csv')
gold.head()

,Tanggal,Terakhir,Pembukaan,Tertinggi,Terendah,Vol.,Perubahan%
0,31/03/2024,1.144.430,1.140.016,1.145.381,1.140.016,NaN,"0,41%"
1,29/03/2024,1.139.801,1.140.304,1.142.120,1.138.473,NaN,"-0,04%"
2,28/03/2024,1.140.304,1.118.933,1.140.806,1.117.077,NaN,"1,91%"
3,27/03/2024,1.118.885,1.109.696,1.122.509,1.106.975,NaN,"0,89%"
4,26/03/2024,1.108.995,1.103.712,1.116.709,306.901,NaN,"0,47%"


# Data Cleaning

In [53]:
df_list = [cpi, btc, ihsg, usd, gold]

In [57]:
for df in df_list:
    print(df.info(verbose=True))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   year_month  100 non-null    object 
 1   CPI         100 non-null    float64
 2   year_code   100 non-null    int32  
 3   month_code  100 non-null    int32  
 4   Year        100 non-null    object 
 5   Month       100 non-null    object 
dtypes: float64(1), int32(2), object(3)
memory usage: 4.0+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2392 entries, 0 to 2391
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Tanggal     2392 non-null   object
 1   Terakhir    2392 non-null   object
 2   Pembukaan   2392 non-null   object
 3   Tertinggi   2392 non-null   object
 4   Terendah    2392 non-null   object
 5   Vol.        2391 non-null   object
 6   Perubahan%  2392 non-null   object
dtypes: object(7)
memory usage: 130.

## Duplicates

In [54]:
for df in df_list:
    print(f'number of duplicate rows: {df.duplicated().sum()}')
    print(f'number of all rows: {len(df)}')

number of duplicate rows: 0
number of all rows: 100
number of duplicate rows: 0
number of all rows: 2392
number of duplicate rows: 0
number of all rows: 1599
number of duplicate rows: 0
number of all rows: 1661
number of duplicate rows: 0
number of all rows: 1106


No duplicate row was found

## Missing value

In [58]:
for df in df_list:
    print(f'number of missing values: {df.isnull().sum()}')

number of missing values: year_month    0
CPI           0
year_code     0
month_code    0
Year          0
Month         0
dtype: int64
number of missing values: Tanggal       0
Terakhir      0
Pembukaan     0
Tertinggi     0
Terendah      0
Vol.          1
Perubahan%    0
dtype: int64
number of missing values: Tanggal       0
Terakhir      0
Pembukaan     0
Tertinggi     0
Terendah      0
Vol.          2
Perubahan%    0
dtype: int64
number of missing values: Tanggal         0
Terakhir        0
Pembukaan       0
Tertinggi       0
Terendah        0
Vol.          671
Perubahan%      0
dtype: int64
number of missing values: Tanggal          0
Terakhir         0
Pembukaan        0
Tertinggi        0
Terendah         0
Vol.          1106
Perubahan%       0
dtype: int64


Missing values were found in non-important columns (all were found in volume columns) so we are just gonna ignore.

## Formatting & change column data type

In [59]:
numerical_columns = ['Terakhir','Pembukaan','Tertinggi','Terendah']

In [60]:
btc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2392 entries, 0 to 2391
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Tanggal     2392 non-null   object
 1   Terakhir    2392 non-null   object
 2   Pembukaan   2392 non-null   object
 3   Tertinggi   2392 non-null   object
 4   Terendah    2392 non-null   object
 5   Vol.        2391 non-null   object
 6   Perubahan%  2392 non-null   object
dtypes: object(7)
memory usage: 130.9+ KB


In [61]:
btc['Terakhir'] = btc['Terakhir'].apply(lambda x: x.replace(".",""))

In [62]:
for df in df_list[1:]:
    df['Tanggal'] = pd.to_datetime(df['Tanggal'],format="%d/%m/%Y")
    df['Vol.'] = 
    for col in numerical_columns:
        df[col] = df[col].apply(lambda x: x.replace('.','').replace(',','.'))
        # df[col] = df[col].apply(lambda x: x.replace('.',''))
        df[col] = df[col].astype('float')

SyntaxError: invalid syntax (1160523163.py, line 3)

In [63]:
btc.head()

,Tanggal,Terakhir,Pembukaan,Tertinggi,Terendah,Vol.,Perubahan%
0,31/03/2024,1129259008,1.113.600.000,1.129.260.032,1.113.600.000,"0,02K","1,41%"
1,30/03/2024,1113600000,1.117.326.976,1.120.000.000,1.113.500.032,"0,02K","-0,28%"
2,29/03/2024,1116734976,1.130.334.976,1.130.855.040,1.105.056.000,"0,03K","-1,20%"
3,28/03/2024,1130334976,1.109.442.944,1.135.718.016,1.100.519.936,"0,04K","1,83%"
4,27/03/2024,1109984000,1.114.294.016,1.130.000.000,1.091.016.960,"0,07K","-0,38%"


In [66]:
btc['Vol.'].unique()

array(['0,02K', '0,03K', '0,04K', '0,07K', '0,09K', '0,06K', '0,11K',
       '0,21K', '0,05K', '0,08K', '0,17K', '0,10K', '0,12K', '0,23K',
       '0,18K', '0,14K', '0,20K', '0,01K', '0,13K', '0,00K', '0,16K',
       '0,15K', '0,32K', '0,19K', '0,33K', '0,25K', '0,40K', nan, '0,28K',
       '0,30K', '0,24K', '0,22K', '0,29K', '0,38K', '0,63K', '0,34K',
       '0,54K', '0,26K', '0,27K', '0,70K', '0,46K', '0,50K', '0,45K',
       '0,42K', '0,97K', '0,39K', '0,61K', '0,69K', '0,59K', '0,35K',
       '0,75K', '0,41K', '0,44K', '0,64K', '0,37K', '0,31K', '0,48K',
       '0,60K', '0,49K', '0,77K', '0,36K', '0,47K', '0,62K', '0,43K',
       '0,76K', '0,85K', '0,52K', '1,09K', '0,83K', '0,57K', '0,53K',
       '0,71K', '1,15K', '0,80K', '0,98K', '1,36K', '1,01K', '0,66K',
       '0,82K', '0,68K', '0,58K', '0,67K', '0,89K', '1,34K', '1,22K',
       '0,81K', '0,55K', '0,93K', '0,72K', '0,51K', '0,56K', '0,65K',
       '0,87K', '0,74K', '1,02K', '0,88K', '0,91K', '0,94K', '0,78K',
       '0,96K',

In [ ]:
y = btc['Vol.'].replace({',':'.',' ':''},regex=False).map(pd.eval).astype(float)

In [ ]:
y

In [ ]:
fix_decimal = btc['Vol.'].apply(lambda x: str(x).replace({',':'.','nan':None},regex=True)

In [ ]:
fix_decimal

In [ ]:
fix_decimal = btc['Vol.'].replace([',','nan'],['.',None])
# fix_decimal = btc['Vol.'].replace({',':'.','nan':None},regex=True)
fix_decimal2 = ihsg['Vol.'].replace({',':'.','nan':None},regex=True)

In [ ]:
fixed2 = fix_decimal2.replace({'K': '*1e3', 'M': '*1e6','B':'*1e9'},regex=True).map(pd.eval)

In [ ]:
fixed = fix_decimal.replace({'K': '*1e3', 'M': '*1e6','B':'*1e9'},regex=True).map(pd.eval)

# EDA & Viz

In [ ]:
gold.info()

In [ ]:
ihsg.head()

In [ ]:
btc.info()

In [ ]:
color_dict = {
    'Bitcoin':'rgb(204,80,62)',
    'IHSG':'rgb(56,166,165)',
    'USD':'rgb(115,175,72)',
    'Gold':'rgb(237,173,8)'
}

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=btc['Tanggal'],y=btc['Terakhir'],mode='lines',name='Bitcoin',line={'color':color_dict['Bitcoin']}))
fig.add_trace(go.Scatter(x=ihsg['Tanggal'],y=ihsg['Terakhir'],mode='lines',name='IHSG',line={'color':color_dict['IHSG']}))
fig.add_trace(go.Scatter(x=usd['Tanggal'],y=usd['Terakhir'],mode='lines',name='USD',line={'color':color_dict['USD']}))
fig.add_trace(go.Scatter(x=gold['Tanggal'],y=gold['Terakhir'],mode='lines',name='Gold',line={'color':color_dict['Gold']}))

fig.show()

As can be seen, Bitcoin prices are significantly higher, resulting in other instruments looking non significant, hence we are going to rescale using logarithmic scale.

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=btc['Tanggal'],y=btc['Terakhir'],mode='lines',name='Bitcoin',line={'color':color_dict['Bitcoin']}))
fig.add_trace(go.Scatter(x=ihsg['Tanggal'],y=ihsg['Terakhir'],mode='lines',name='IHSG',line={'color':color_dict['IHSG']}))
fig.add_trace(go.Scatter(x=usd['Tanggal'],y=usd['Terakhir'],mode='lines',name='USD',line={'color':color_dict['USD']}))
fig.add_trace(go.Scatter(x=gold['Tanggal'],y=gold['Terakhir'],mode='lines',name='Gold',line={'color':color_dict['Gold']}))
fig.update_yaxes(type='log')

fig.show()

In [5]:
hostname = 'localhost'
database = 'assets_db'
username = 'postgres'
pwd = '122126'
port_id = 5432

conn = psycopg2.connect(host=hostname,dbname=database,user=username,password=pwd,port=port_id)
cur = conn.cursor()



In [10]:
query = """
    CREATE TABLE IF NOT EXISTS cpi (
        year_month    INTEGER,
        CPI           DECIMAL,
        year_code     INTEGER,
        month_code    INTEGER,
        year          INTEGER,
        month         VARCHAR(20)
    );
    """

cur.execute(query)
conn.commit()

In [2]:
def get_monthly_cpi(url):
    api_key = API_key
    url = f'{url}/{api_key}'
    response = requests.get(url)
    response.raise_for_status()
    fetched = response.json()
    fixed = json.dumps(fetched)
    data = json.loads(fixed)
    
    vervar = next(item['val'] for item in data['vervar'] if item['label'].upper() == 'INDONESIA')
    var = next(item['val'] for item in data['var'])
    turvar = next(item['val'] for item in data['turvar'])
    code = str(vervar)+str(var)+str(turvar)
    
    datacontent = data['datacontent']
    year_mapping = {item['val']:item['label'] for item in data['tahun']}
    month_mapping = {item['val']:item['label'] for item in data['turtahun']}
    last_four_years = [key for key in year_mapping.keys() if key >= max(year_mapping)-3]   
    filter_keys = [i+str(num) for num in range(1,13) for i in [code + str(num) for num in last_four_years]]
    filtered = {key: value for key, value in datacontent.items() if key in filter_keys}
    df = pd.DataFrame.from_dict(filtered,orient='index').reset_index().rename(columns={'index':'year_month',0:'CPI'})
    
    df['year_month'] = df['year_month'].apply(lambda x: x[len(code):])
    df['year_code'] = df['year_month'].str[:3].astype('int32')
    df['month_code'] = df['year_month'].str[3:].astype('int32')
    df['Year'] = df['year_code'].map(year_mapping)
    df['Month'] = df['month_code'].map(month_mapping)
    return df

In [9]:
def insert_to_table(conn,dataframe,table_name):
    try:
        dataframe.to_sql(name=table_name,con=engine,if_exists='replace',index=False)
        # conn.autocommit = True
    except Exception as e:
        print(f"Error: {e}")
    else:
        conn.commit()
        print(f"{table_name} has been inserted")

In [12]:
def init_connection():
    # create connection
    conn = psycopg2.connect(**param_database,options='-csearch_path=dbo,dev')
    # create cursor
    cur = conn.cursor()
    conn_string = f"postgresql://{param_database['user']}:{param_database['password']}@{param_database['host']}/{param_database['dbname']}?options=-csearch_path%3Ddbo,dev"
    engine = create_engine(conn_string).connect()
    return conn, cur, engine

In [16]:
param_database = config('postgresql')

In [17]:
conn, cur, engine = init_connection()

In [4]:
df1 = get_monthly_cpi('https://webapi.bps.go.id/v1/api/list/model/data/lang/ind/domain/0000/var/2/key')

In [7]:
cpi_links = ['https://webapi.bps.go.id/v1/api/list/model/data/lang/ind/domain/0000/var/2/key',
            'https://webapi.bps.go.id/v1/api/list/model/data/lang/ind/domain/0000/var/1709/key',
            'https://webapi.bps.go.id/v1/api/list/model/data/lang/ind/domain/0000/var/2261/key']

In [8]:
cpi_list = [get_monthly_cpi(link) for link in cpi_links]
cpi = pd.concat(cpi_list,ignore_index=True)

In [18]:
insert_to_table(conn,cpi,'cpi')

cpi has been inserted


In [84]:
type(cpi)

pandas.core.frame.DataFrame

In [19]:
cur.close()
conn.close()

In [102]:
import pandas as pd
import numpy as np
import datetime as dt
import requests
import json
import psycopg2
from config import config
from sqlalchemy import create_engine

API_key = config('BPS_API')['api_key']
param_database = config('postgresql')
csv_files = config('csv_directory')

def get_monthly_cpi(url):
    api_key = API_key
    url = f'{url}/{api_key}'
    response = requests.get(url)
    response.raise_for_status()
    if response.status_code != 204:
        fetched = response.json()
    fixed = json.dumps(fetched)
    data = json.loads(fixed)
    
    vervar = next(item['val'] for item in data['vervar'] if item['label'].upper() == 'INDONESIA')
    var = next(item['val'] for item in data['var'])
    turvar = next(item['val'] for item in data['turvar'])
    code = str(vervar)+str(var)+str(turvar)
    
    datacontent = data['datacontent']
    year_mapping = {item['val']:item['label'] for item in data['tahun']}
    month_mapping = {item['val']:item['label'] for item in data['turtahun']}
    last_four_years = [key for key in year_mapping.keys() if key >= max(year_mapping)-3]   
    filter_keys = [i+str(num) for num in range(1,13) for i in [code + str(num) for num in last_four_years]]
    filtered = {key: value for key, value in datacontent.items() if key in filter_keys}
    df = pd.DataFrame.from_dict(filtered,orient='index').reset_index().rename(columns={'index':'year_month',0:'CPI'})
    
    df['year_month'] = df['year_month'].apply(lambda x: x[len(code):])
    df['year_code'] = df['year_month'].str[:3].astype('int32')
    df['month_code'] = df['year_month'].str[3:].astype('int32')
    df['Year'] = df['year_code'].map(year_mapping)
    df['Month'] = df['month_code'].map(month_mapping)
    return df

In [103]:
df1 = get_monthly_cpi('https://webapi.bps.go.id/v1/api/list/model/data/lang/ind/domain/0000/var/2/key')
# cpi_links = ['https://webapi.bps.go.id/v1/api/list/model/data/lang/ind/domain/0000/var/2/key',
#             'https://webapi.bps.go.id/v1/api/list/model/data/lang/ind/domain/0000/var/1709/key',
#             'https://webapi.bps.go.id/v1/api/list/model/data/lang/ind/domain/0000/var/2261/key']

# cpi_list = [get_monthly_cpi(link) for link in cpi_links]
# cpi = pd.concat(cpi_list,ignore_index=True,axis=0)
# print(type(cpi))

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [106]:
import os

print(os.getcwd())

file_path = os.path.join(os.getcwd(), "data.json")
print(os.path.exists(file_path))
print(os.stat(file_path))

C:\Users\Windows10\Documents\bitcoin_project
False


FileNotFoundError: [WinError 2] The system cannot find the file specified: 'C:\\Users\\Windows10\\Documents\\bitcoin_project\\data.json'